In [1]:
import os
import sys
import tracemalloc

In [2]:
import pandas as pd

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [5]:
print('VIRTUAL_ENV =',os.environ.get('VIRTUAL_ENV'))
print('CONDA_DEFAULT_ENV =',os.environ.get('CONDA_DEFAULT_ENV'))
print('PYTHONTRACEMALLOC =',os.environ.get('PYTHONTRACEMALLOC'))
print('PYTHONPATH =',os.environ.get('PYTHONPATH'))
print()
print('sys.path = ',sys.path)
print('sys.executable = ',sys.executable)

VIRTUAL_ENV = None
CONDA_DEFAULT_ENV = None
PYTHONTRACEMALLOC = 1
PYTHONPATH = /work

sys.path =  ['/work/notebooks', '/work', '/opt/conda/lib/python311.zip', '/opt/conda/lib/python3.11', '/opt/conda/lib/python3.11/lib-dynload', '', '/opt/conda/lib/python3.11/site-packages']
sys.executable =  /opt/conda/bin/python


In [6]:
# Включаем трассировку
tracemalloc.start()

In [7]:
def total_memory_allocated(verbose=True):
    # Получаем текущую статистику использования памяти
    snapshot = tracemalloc.take_snapshot()
    top_stats = snapshot.statistics('lineno')
    # Вычисляем общий объем аллоцированной памяти
    total_memory_allocated = sum(stat.size for stat in top_stats)
    if verbose:
        print("Общий объем аллоцированной памяти:", '{:,}'\
              .format(total_memory_allocated).replace(',', '\''), "байт")
    return total_memory_allocated

In [8]:
tma = total_memory_allocated()

Общий объем аллоцированной памяти: 81'641'835 байт


In [9]:
# Функция отображения прироста объема аллоцированной памяти
def tmach(tma):
    if not tma:
        tma = 0
    tma1 = total_memory_allocated(verbose=False)
    print("Прирост:", '{:,}'\
    .format(tma1-tma).replace(',', '\''), "байт")
    return tma1

Testing operations_1_-0-

#### Точка входа = Приложение с использованием SparkContext

In [10]:
sc = SparkContext(appName="appName")
weather_entry = sc.parallelize(['2009-01-01', 15.1,26.1])
print(weather_entry.take(3))

['2009-01-01', 15.1, 26.1]


In [11]:
tma = tmach(tma)

Прирост: 1'063'314 байт


In [12]:
sc.stop()

#### Точка входа = Приложение с использованием SparkSession

Данный способ является болеее предпочтительным в новых версиях по сравненеию с SparkContext

In [13]:
APP_NAME = 'sampleApp'
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
# Вывести свойства объекта spark
print("AppName:", spark.sparkContext.appName)
print("Master:", spark.sparkContext.master)
print("Version:", spark.version)
# Другие свойства, которые вы хотите вывести...

AppName: sampleApp
Master: local[*]
Version: 3.5.0


In [14]:
tma = tmach(tma)

Прирост: 163'740 байт


In [15]:
spark.stop()

#### Создание сессии с явным указанием SPARK_MASTER_IP

In [16]:
# from pyspark.sql import SparkSession
# # Вводим Spark Master IP, который выдаст нам скрипт 
# SPARK_MASTER_IP = '172.18.0.2' 
# spark = SparkSession.builder.appName("pyspark-taxi-forecasting") \
#     .master(f"spark://{SPARK_MASTER_IP}:7077") \
#     .config("spark.executor.cores", 1) \
#     .config("spark.task.cpus", 1) \
#     .getOrCreate()
## .config("spark.executor.memory", "1g") \

#### Запуск сессии с использованием переменной окружения SPARK_MASTER_IP.

При создании кластера Спарк предполагается определение значения Спарк-мастер IP и его установка в переменную окружения контейнера jupyter_lab

In [17]:
# Получаем значение переменной окружения SPARK_MASTER_IP
spark_master_ip = os.getenv('SPARK_MASTER_IP')

In [18]:
# Если переменная окружения установлена, используем ее для настройки Spark Session
if spark_master_ip:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("pyspark-taxi-forecasting") \
        .master(f"spark://{spark_master_ip}:7077") \
        .config("spark.executor.cores", "1") \
        .config("spark.task.cpus", 1) \
        .getOrCreate()
    print(f"Spark Master IP: {spark_master_ip}")
else:
    print("Переменная окружения SPARK_MASTER_IP не установлена")

Spark Master IP: 172.18.0.2


In [19]:
tma = tmach(tma)

Прирост: 26'700 байт


In [20]:
# Вывести свойства объекта spark
print("AppName:", spark.sparkContext.appName)
print("Master:", spark.sparkContext.master)
print("Version:", spark.version)

AppName: pyspark-taxi-forecasting
Master: spark://172.18.0.2:7077
Version: 3.5.0


### Загрузка данных

In [21]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2023.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 27.3 ms, sys: 12.1 ms, total: 39.4 ms
Wall time: 47.8 s


In [22]:
tma = tmach(tma)

Прирост: -10'542 байт


In [23]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [24]:
# Отобразить первые несколько строк DataFrame
# taxi.show()

#### Загрузка таблицы с описанием полей данных

In [25]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';')

In [26]:
table_descr

,Column Name,Description,Type,Описание,Тип
0,Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст
1,Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст
2,Trip Start Timestamp,When the trip started rounded to the nearest 1...,Date & Time,Время начала поездки округленное до ближайших...,Дата и время
3,Trip End Timestamp,When the trip ended rounded to the nearest 15 ...,Date & Time,Время окончания поездки округленное до ближай...,Дата и время
4,Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число
5,Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число
6,Pickup Census Tract,The Census Tract where the trip began. For pri...,Plain Text,Код района по переписи населения где началас...,Обычный текст
7,Dropoff Census Tract,The Census Tract where the trip ended. For pri...,Plain Text,Код района по переписи населения где заверши...,Обычный текст
8,Pickup Community Area,The Community Area where the trip began. This ...,Number,Номер коммунальной области где началась поез...,Число
9,Dropoff Community Area,The Community Area where the trip ended. This ...,Number,Номер коммунальной области где завершилась п...,Число


In [27]:
# taxi[["Trip ID", "Taxi ID", "Trip Miles","Trip Start Timestamp","Trip End Timestamp","Trip Seconds","Trip Miles","Pickup Census Tract"]].show(5)

In [ ]:
# taxi[["Trip ID", "Dropoff Census Tract", "Pickup Community Area","Dropoff Community Area","Trip End Timestamp","Trip Total","Trip Miles","Company"]].show(5)

In [ ]:
# taxi[["Trip ID", "Pickup Centroid Latitude", "Pickup Centroid Longitude","Pickup Centroid Location",
#       "Dropoff Centroid Latitude","Dropoff Centroid Longitude","Dropoff Centroid  Location"]].show(5)

In [27]:
%%time
# print(taxi.describe().show())
# Запустите метод describe() на вашем PySpark DataFrame (например, taxi)
describe_result = taxi.describe().toPandas()

CPU times: user 403 ms, sys: 88.7 ms, total: 492 ms
Wall time: 1min 36s


In [28]:
describe_result

,summary,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,count,3783730,3783730,3783730,3783682,3783012,3783717,1650232,1617623,3615963,...,3778327,3778327,3783730,3783730,3617351,3617351,3617351,3441935,3441935,3441935
1,mean,None,None,None,None,1235.3278387697421,6.471233194766421,1.7031501647110167E10,1.7031414613682808E10,35.03091652209937,...,2.241853884007393,27.371134848836103,None,None,41.90200067412448,-87.69906175284666,None,41.89419484402875,-87.66236339478645,None
2,stddev,None,None,None,None,1736.5661018295184,7.593310441212217,373503.064706103,344107.3452521375,26.0787201470195,...,19.47903838213312,37.03373571024758,None,None,0.06251123711511888,0.11213128488994714,None,0.05656711663584078,0.07345541720850372,None
3,min,0000012deb83dbb55726d5a75c374197d0641fa0,00110971c7c4a7173fcf93f49a22d6b9b0a02c27c4b9f8...,01/01/2023 01:00:00 AM,01/01/2023 01:00:00 AM,0,0.0,17031010100,17031010100,1,...,0.0,0.0,Cash,2733 - 74600 Benny Jona,41.651921576,-87.913624596,POINT (-87.5313862567 41.7204632831),41.660136051,-87.913624596,POINT (-87.5349029012 41.707311449)
4,max,fffffe03acfa1552c98fad12d73ff0aca70a5c2a,ffd231d2536b9463d888cfbb42f36d543b37d22d96a6dd...,08/01/2023 12:00:00 AM,10/17/2022 10:00:00 AM,86340,945.4,17031980100,17031980100,77,...,9446.65,9999.75,Unknown,U Taxicab,42.021223593,-87.531386257,POINT (-87.913624596 41.9802643146),42.021223593,-87.534902901,POINT (-87.913624596 41.9802643146)


In [29]:
describe_result = pd.read_csv('/work/data/2023_describe.csv',sep=',')

In [30]:
print(describe_result.transpose())

                                    0                   1              2  \
summary                         count                mean         stddev   
Trip ID                       3783730                 NaN            NaN   
Taxi ID                       3783730                 NaN            NaN   
Trip Start Timestamp          3783730                 NaN            NaN   
Trip End Timestamp            3783682                 NaN            NaN   
Trip Seconds                3783012.0         1235.327839    1736.566102   
Trip Miles                  3783717.0            6.471233        7.59331   
Pickup Census Tract         1650232.0  17031501647.110168  373503.064706   
Dropoff Census Tract        1617623.0  17031414613.682808  344107.345252   
Pickup Community Area       3615963.0           35.030917       26.07872   
Dropoff Community Area      3419046.0           26.097846      20.917892   
Fare                        3778327.0            21.99552      22.233947   
Tips        

In [ ]:
# output_file_path = '/work/2023_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [ ]:
# taxi.registerTempTable("taxi")
"""
/usr/local/spark/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
"""
taxi.createOrReplaceTempView("taxi")

In [ ]:
%%time
# вам нужно использовать обратные кавычки ` ` (backticks) или просто имя столбца без кавычек
print(spark.sql("SELECT COUNT(*) FROM taxi WHERE `Trip Seconds` < 300").show())

In [ ]:
from pyspark.sql.functions import col, dayofweek, to_timestamp, rand, substring

In [ ]:
def random_sample_dataframe(dataframe, percentage):
    # Генерируем случайные числа от 0 до 1 и фильтруем строки
    taxis = dataframe.filter(rand() < percentage)
    
    return taxis

In [ ]:
taxis = random_sample_dataframe(taxi, 0.05)

In [ ]:
taxis[["Trip ID", "Trip Start Timestamp"]].show(5)

In [ ]:
def determine_date_format(dataframe):
    # Выбираем первые два символа из столбца "Trip Start Timestamp"
    first_two_chars = dataframe.select(substring("Trip Start Timestamp", 1, 2))
    
    # Подсчитываем уникальные значения первых двух символов
    unique_values = first_two_chars.distinct().count()
    
    # Определяем маску на основе количества уникальных значений
    date_format = "dd/MM/yyyy" if unique_values >= 13 else "MM/dd/yyyy"
    
    return date_format, unique_values

In [ ]:
%%time
# Пример использования:
date_format, unique_values = determine_date_format(taxis)
print("Дата формат:", date_format,", количество уникальных значений в 1-х двух знаках:",unique_values)

In [ ]:
taxis.select(substring("Trip Start Timestamp", 1, 2)).distinct().show()

In [ ]:
taxis[["Trip Start Timestamp"]].show(n=15, truncate=False)

In [ ]:
taxis.select(to_timestamp(taxis["Trip Start Timestamp"], 'MM/dd/yyyy hh:mm:ss a').alias("Trip Start Timestamp")).show(15)

In [ ]:
taxis = taxis.withColumn("Trip Start Timestamp", to_timestamp(taxis["Trip Start Timestamp"], 'MM/dd/yyyy hh:mm:ss a'))

In [ ]:
taxis[["Trip Start Timestamp"]].show(n=15, truncate=False)

In [ ]:
taxis = taxis.withColumn("DayOfWeek", dayofweek(col("Trip Start Timestamp")))

In [ ]:
%%time
# taxis.createOrReplaceTempView("taxis")
q = \
"""
SELECT
    DayOfWeek,
    AVG(`Trip Seconds`)
FROM taxis
GROUP BY DayOfWeek
"""
print(spark.sql(q).show(10))

Вывод %%time в Jupyter Notebook содержит информацию о времени выполнения кода в двух разных контекстах:

    CPU times: Это время, которое ваш код фактически проводит в CPU (Central Processing Unit) или процессоре. Он разбивается на две части:
        user: Это время, в течение которого CPU выполняет пользовательский код (ваш код).
        sys: Это время, которое CPU тратит на выполнение системных операций, таких как ввод/вывод данных.

    Wall time: Это общее время, которое прошло с начала выполнения кода до его завершения. Это включает в себя не только вычислительное время, но и время, которое может быть затрачено на ожидание ввода/вывода, блокировку потоков и другие факторы, которые могут вызвать задержки в выполнении кода.

В вашем конкретном выводе:

    user составляет 10.5 миллисекунд (0.0105 секунд).
    sys составляет 2.63 миллисекунд (0.00263 секунд).
    total (сумма user и sys) равно 13.2 миллисекунд (0.0132 секунд).

Это время, которое фактически затратилось на выполнение кода на процессоре.

    Wall time составляет 25.3 секунды. Это общее время с момента запуска ячейки до ее завершения, включая все вычисления и возможные задержки.

Обратите внимание, что Wall time (время на стенке) обычно больше, чем сумма user и sys, потому что оно включает в себя также время, которое может быть затрачено на ожидание ввода/вывода или другие факторы, которые могут замедлить выполнение кода.

In [ ]:
taxis.printSchema()

In [ ]:
taxis[["Trip ID", "Trip Start Timestamp","DayOfWeek"]].show(5)